In [1]:
from tqdm import tqdm
import numpy as np
from utils import *
import numpy as np
import math as m
import matplotlib.pyplot as plt
import os
import math as m
from glob import glob
import haversine as hs 
from scipy import signal

# script requires 30GB RAM


In [2]:
GRIDSIZE = 800 # should be above 300, else in becomes inaccurate
RADIUS = 300 # radius in m
BLUR = 4 # by how many pixels the populationgrid should be blurred

In [3]:
all_city_paths = glob("data/*")
all_city_paths.pop(0)
population_df = load_us_population_df()

100%|██████████| 6/6 [02:01<00:00, 20.23s/it]


In [17]:
result_list = []

def get_data(city_name):
    stops_df = load_stops(os.path.basename(city_name))

    # skip if no stops
    if len(stops_df) <= 1000:
        raise ValueError
        

    minlat = stops_df["stop_lat"].min()
    maxlat = stops_df["stop_lat"].max()
    minlon = stops_df["stop_lon"].min()
    maxlon = stops_df["stop_lon"].max()

    # select rows that are inside of the bounding box
    selected_df = population_df[(population_df['x'] > minlat) & (population_df['x'] < maxlat) & (population_df['y'] > minlon) & (population_df['y'] < maxlon)]


    # convert to numpy
    xlist = selected_df["x"].tolist()
    ylist = selected_df["y"].tolist()
    weight = selected_df["weight"].tolist()

    # init grids on which calculations will happen
    POPULATIONGRID = [[0 for i in range(GRIDSIZE)] for j in range (GRIDSIZE)]
    STOPSGRID = [[0 for i in range(GRIDSIZE)] for j in range (GRIDSIZE)]

    # calculate stepsize for normalisation
    lat_step = (maxlat - minlat) / GRIDSIZE
    lon_step = (maxlon - minlon) / GRIDSIZE

    # normalize in 0 - GRIDSIZE Range and Place Population on the grid
    for x, y, w in zip(xlist, ylist, weight):
        row = int((x - minlat) // lat_step)
        col = int((y - minlon) // lon_step)
        if row <= GRIDSIZE and row >=0 and col <= GRIDSIZE and col >=0 :
            POPULATIONGRID[row][col] += int(w)

    # Blurr Population, while keeping the Total sum the same. If this is not acounted for, the population will fluctuate by a lot
    kernel = [[1/(BLUR*BLUR)]*BLUR for _ in range(BLUR)]  # Define blurring kernel

    # Initialize output list
    blurred_list =np.zeros((GRIDSIZE, GRIDSIZE))  

    # Perform convolution (BLUR) operation
    for i in range(BLUR//2, GRIDSIZE-BLUR//2):
        for j in range(BLUR//2, GRIDSIZE-BLUR//2):

            patch = [POPULATIONGRID[x][j-BLUR//2:j+BLUR//2+1] for x in range(i-BLUR//2, i+BLUR//2+1)]
            patch_sum = np.sum(patch)

            blurred_list[i][j] = patch_sum * kernel[BLUR//2][BLUR//2]

    # list is now blurred
    POPULATIONGRID = blurred_list

    # calculate diagonal and side length of the rectangle over all the stops
    diagonal = hs.haversine((maxlat, maxlon), (minlat, minlon))
    side = diagonal / m.sqrt(2)

    # calculate realworld scale
    onepxlength = side / GRIDSIZE # length of one pixel in grid in KM

    # calculate the row and col where the stop- pixel goes 
    stops_df["row"] = (stops_df["stop_lat"] - minlat) // lat_step
    stops_df["col"] = (stops_df["stop_lon"] - minlon) // lon_step

    # TODO make this NAN safe
    # convert to integer and then to list
    rowlist = stops_df["row"].astype(int).tolist()
    collist = stops_df["col"].astype(int).tolist()

    # loop over list and set pixel on the grid
    for r, c in zip(rowlist, collist):
        if r < GRIDSIZE and c < GRIDSIZE:
            STOPSGRID[r][c] += 1

    # calculate radius in px
    radius_px = m.floor(RADIUS / (onepxlength * 1000)) + 1

    # define grid where a circle will be drawn for convoluting it later
    circlegrid =  [[0 for i in range(radius_px * 2 + 1)] for j in range (radius_px * 2 + 1)]

    # draw circle with radius on circle grid
    for x in range(radius_px * 2 + 1):
        for y in range(radius_px * 2 + 1):
            deltax = radius_px - x
            deltay = radius_px - y

            d = m.sqrt((deltax * deltax) + (deltay * deltay))

            if d <= radius_px:
                circlegrid[x][y] = 1
            else:
                circlegrid[x][y] = 0

    # convolve circle over stopsgrid
    CONVOLVED = signal.fftconvolve(STOPSGRID, circlegrid, mode = "same")
    # remove rounding errors and set every pixel greater than 0 to 1
    CONVOLVED = np.where(CONVOLVED>0 + 10e-2, 1, 0)

    #plt.pcolormesh(CONVOLVED)
    #plt.show()

    # get final sum
    people = 0
    for x in range(GRIDSIZE):
        for y in range(GRIDSIZE):
            if (CONVOLVED[x][y] >= 1):
                people += POPULATIONGRID[x][y]

    
    totalpop = np.sum(POPULATIONGRID)
    percentage = (people / totalpop) * 100

    #print(os.path.basename(city_name), " total:", m.floor(totalpop), " positives:" , m.floor(people), m.floor(percentage), "%", " stops:", len(stops_df))

    result_list.append([city_name,totalpop,people,percentage,len(stops_df),side,minlat,maxlat,minlon,maxlon,GRIDSIZE,RADIUS])
    print(result_list)


for city_name in tqdm(all_city_paths):
    try:
        get_data(city_name)
    except Exception as e:
        pass

res_df = pd.DataFrame(result_list, columns = ["name","population","positives","percentage","stops","side_length","minlat","maxlat","minlon","maxlon","gridsize","radius"])
print(res_df.head())

res_df.to_csv("Umkreisdata.csv")

  0%|          | 1/400 [00:19<2:12:25, 19.91s/it]

[['data\\US_Alabama_Birmingham', 638308.625, 170916.9375, 26.77653580194064, 1317, 32.249551689403326, 33.36195, 33.66285, -87.011263, -86.67695999999998, 800, 300]]


  1%|          | 3/400 [00:39<1:22:41, 12.50s/it]

[['data\\US_Alabama_Birmingham', 638308.625, 170916.9375, 26.77653580194064, 1317, 32.249551689403326, 33.36195, 33.66285, -87.011263, -86.67695999999998, 800, 300], ['data\\US_Alabama_Montgomery', 255515.9375, 120737.875, 47.25258086885481, 1841, 21.198375912493592, 32.293826, 32.4342702556604, -86.4007303, -86.128268729575, 800, 300]]


  8%|▊         | 31/400 [01:01<07:20,  1.19s/it] 

...

[['data\\US_Alabama_Birmingham', 638308.625, 170916.9375, 26.77653580194064, 1317, 32.249551689403326, 33.36195, 33.66285, -87.011263, -86.67695999999998, 800, 300], ['data\\US_Alabama_Montgomery', 255515.9375, 120737.875, 47.25258086885481, 1841, 21.198375912493592, 32.293826, 32.4342702556604, -86.4007303, -86.128268729575, 800, 300], ['data\\US_Arizona_Phoenix', 5891810.1875, 2604321.1875, 44.202394588768314, 8017, 133.28227516461178, 32.382706, 33.740293, -112.872185, -111.660913, 800, 300], ['data\\US_California_Fresno', 899286.5625, 481769.9375, 53.57246039134495, 1491, 21.872923443797493, 36.681009, 36.888538, -119.908969, -119.677657, 800, 300], ['data\\US_California_Hesperia', 1897356.5625, 287598.8125, 15.157868488411754, 1018, 115.83180783887632, 34.072411, 35.273818, -117.64669, -116.609954, 800, 300], ['data\\US_California_Riverside', 7475849.0, 1046816.0625, 14.002637860930578, 2382, 84.92200887003814, 33.452304, 34.10009, -117.915548, -116.875799, 800, 300], ['data\\US_C

 97%|█████████▋| 389/400 [26:32<00:42,  3.91s/it]

[['data\\US_Alabama_Birmingham', 638308.625, 170916.9375, 26.77653580194064, 1317, 32.249551689403326, 33.36195, 33.66285, -87.011263, -86.67695999999998, 800, 300], ['data\\US_Alabama_Montgomery', 255515.9375, 120737.875, 47.25258086885481, 1841, 21.198375912493592, 32.293826, 32.4342702556604, -86.4007303, -86.128268729575, 800, 300], ['data\\US_Arizona_Phoenix', 5891810.1875, 2604321.1875, 44.202394588768314, 8017, 133.28227516461178, 32.382706, 33.740293, -112.872185, -111.660913, 800, 300], ['data\\US_California_Fresno', 899286.5625, 481769.9375, 53.57246039134495, 1491, 21.872923443797493, 36.681009, 36.888538, -119.908969, -119.677657, 800, 300], ['data\\US_California_Hesperia', 1897356.5625, 287598.8125, 15.157868488411754, 1018, 115.83180783887632, 34.072411, 35.273818, -117.64669, -116.609954, 800, 300], ['data\\US_California_Riverside', 7475849.0, 1046816.0625, 14.002637860930578, 2382, 84.92200887003814, 33.452304, 34.10009, -117.915548, -116.875799, 800, 300], ['data\\US_C

100%|██████████| 400/400 [26:52<00:00,  4.03s/it]

[['data\\US_Alabama_Birmingham', 638308.625, 170916.9375, 26.77653580194064, 1317, 32.249551689403326, 33.36195, 33.66285, -87.011263, -86.67695999999998, 800, 300], ['data\\US_Alabama_Montgomery', 255515.9375, 120737.875, 47.25258086885481, 1841, 21.198375912493592, 32.293826, 32.4342702556604, -86.4007303, -86.128268729575, 800, 300], ['data\\US_Arizona_Phoenix', 5891810.1875, 2604321.1875, 44.202394588768314, 8017, 133.28227516461178, 32.382706, 33.740293, -112.872185, -111.660913, 800, 300], ['data\\US_California_Fresno', 899286.5625, 481769.9375, 53.57246039134495, 1491, 21.872923443797493, 36.681009, 36.888538, -119.908969, -119.677657, 800, 300], ['data\\US_California_Hesperia', 1897356.5625, 287598.8125, 15.157868488411754, 1018, 115.83180783887632, 34.072411, 35.273818, -117.64669, -116.609954, 800, 300], ['data\\US_California_Riverside', 7475849.0, 1046816.0625, 14.002637860930578, 2382, 84.92200887003814, 33.452304, 34.10009, -117.915548, -116.875799, 800, 300], ['data\\US_C